### Physical Interpretation of Best Analytic Models

Taking only the PySR schemes from the Pareto frontier and their modified version that *always* satisfies the RH-constraint.

In [1]:
# Ran with Python 3 environment
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import os
import sys
import sympy as sp

sys.path.insert(0, '~/workspace_icon-ml/cloud_cover_parameterization')

import tensorflow as tf
import my_classes
import time
import json

from my_classes import read_mean_and_std
from tensorflow.keras.models import load_model
from tensorflow import nn

abspath = '~/workspace_icon-ml/symbolic_regression/evaluate_schemes/analyzing_eqns/v1/'

CB_color_cycle = ['#377eb8', '#ff7f00', '#4daf4a',
                  '#f781bf', '#a65628', '#984ea3',
                  '#999999', '#e41a1c', '#dede00']

matplotlib.use('PDF')

In [2]:
# Load data
base_path = '/home/b/b309170'
output_path = base_path + '/my_work/icon-ml_data/cloud_cover_parameterization/neighborhood_based_SR_DYAMOND'

input_data = np.load(output_path + '/cloud_cover_input_dyamond.npy')
output_data = np.load(output_path + '/cloud_area_output_dyamond.npy')

# To locate variables
features = ['hus', 'clw', 'cli', 'ta', 'pa', 'zg', 'fr_land', 'U', 'rh', 'ps', 'hus_z', 'hus_zz', 'clw_z', 'clw_zz', 'cli_z',\
                  'cli_zz', 'ta_z', 'ta_zz', 'pa_z', 'pa_zz', 'U_z', 'U_zz', 'rh_z', 'rh_zz']
len(features)

loc = {}
for i in range(len(features)):
    loc[features[i]] = i

In [3]:
# Run with random subset (the entire dataset takes too much memory)
np.random.seed(10)
rand_int = np.random.randint(0, input_data.shape[0], 10**8)

input_data = input_data[rand_int]
output_data = output_data[rand_int]

In [4]:
# Scale data
mean,std = read_mean_and_std('~/workspace_icon-ml/cloud_cover_parameterization/neighborhood_based_SR_DYAMOND/saved_models/cross_validation_neighborhood_based_sr_cl_area_fold_2.txt')

input_data_scaled = (input_data - mean)/std

all_features = ['hus', 'clw', 'cli', 'ta', 'pa', 'zg', 'fr_land', 'U', 'rh', 'ps', 'hus_z', 'hus_zz', 'clw_z', 'clw_zz', 'cli_z', 'cli_zz', 'ta_z', 'ta_zz', 'pa_z', 'pa_zz', 'U_z', 'U_zz', 'rh_z', 'rh_zz']

loc_all = {}
for i in range(len(all_features)):
    loc_all[all_features[i]] = i

In [5]:
# Best PySR equations
with open('~/workspace_icon-ml/symbolic_regression/finding_symmetries/pysr_results_dyamond_on_regimes/no_of_regimes_2/optimized_eqns.json', 'r') as file:
    pysr_eqns = json.load(file)
    
rh, ta, rh_z, cli, clw = sp.symbols('rh ta rh_z cli clw')
x0, x1, x2, x3, x4 = sp.symbols('x0 x1 x2 x3 x4')
    
pysr_EQ1 = sp.lambdify((rh, ta, rh_z, cli, clw), pysr_eqns['EQ1']['Equation w.r.t. physical vars'])
pysr_EQ4 = sp.lambdify((rh, ta, rh_z, cli, clw), pysr_eqns['EQ4']['Equation w.r.t. physical vars'])

def pysr_EQ1_mod(rh, ta, rh_z, cli, clw):
    # Artificially increase RH to ensure RH-constraint
    (a,b,c,d) = (38.85954116, 42.70818472, 19.34746465, 1.11321032)
    
    x0 = (rh - mean[loc_all['rh']])/std[loc_all['rh']]
    x1 = (ta - mean[loc_all['ta']])/std[loc_all['ta']]
    
    x0 = np.maximum(x0, 1/(2*c*d)*(-c*x1**2-a))
    
    rh = x0*std[loc_all['rh']] + mean[loc_all['rh']]
    
    return pysr_EQ1(rh, ta, rh_z, cli, clw)    
    
def pysr_EQ4_mod(rh, ta, rh_z, cli, clw):
    # Artificially increase RH to ensure RH-constraint
    (a,b,c,d) = (38.6562122, 43.53500518, 19.78403208, 1.13637902)
    
    x0 = (rh - mean[loc_all['rh']])/std[loc_all['rh']]
    x1 = (ta - mean[loc_all['ta']])/std[loc_all['ta']]
    
    x0 = np.maximum(x0, 1/(2*c*d)*(-c*x1**2-a))
    
    rh = x0*std[loc_all['rh']] + mean[loc_all['rh']]
    
    return pysr_EQ4(rh, ta, rh_z, cli, clw)  

In [6]:
# Load NN
custom_objects = {}
custom_objects['leaky_relu'] = nn.leaky_relu

# SFS 24 (from workspace/cloud_cover_parameterization/neighborhood_based_SR_DYAMOND/saved_models/cross_validation_neighborhood_based_sr_cl_area_fold_2.h5)
sfs_24_nn = load_model('~/workspace_icon-ml/cloud_cover_parameterization/neighborhood_based_SR_DYAMOND/saved_models/cross_validation_neighborhood_based_sr_cl_area_fold_2.h5', custom_objects)

def b(x):
    return np.minimum(np.maximum(x, 0), 100)

2023-03-13 14:38:55.916245: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-13 14:38:55.917797: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-13 14:38:55.918401: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (l40054.lvt.dkrz.de): /proc/driver/nvidia/version does not exist
2023-03-13 14:38:55.928255: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
# Add data PDPs to the plots!
# Runs for 70s*len(X_vals)
def find_closest_samples(X, X_vals, N):
    '''
        X [str]: The feature, for which we should not use the mean
        X_vals [list(float)]: The values for the scaled! features (instead of the mean)
        N [int]: The number of closest samples, that shall be in the output
        returns: The indices of the N closest samples
    '''
    indices = np.zeros((input_data_scaled.shape[0], len(X_vals)))
    
    feats = ['rh', 'ta', 'rh_z', 'cli', 'clw']
    feats.remove(X)
    feat_inds = [loc[feat] for feat in feats]
    s = np.sum(np.abs(input_data_scaled[:, feat_inds]), axis=1)
    
    for k in range(len(X_vals)):
        indices[:, k] = s + np.abs(input_data_scaled[:, loc[X]] - X_vals[k])
        
    sorted_inds = np.argsort(indices, axis=0) # Takes most of the time
    return sorted_inds[:N]

In [8]:
ta_mean = np.mean(input_data[:, loc['ta']])
rh_z_mean = np.mean(input_data[:, loc['rh_z']])
cli_mean = np.mean(input_data[:, loc['cli']])
clw_mean = np.mean(input_data[:, loc['clw']])
rh_mean = np.mean(input_data[:, loc['rh']])

ta_std = np.mean(input_data[:, loc['ta']])
rh_z_std = np.mean(input_data[:, loc['rh_z']])
cli_std = np.mean(input_data[:, loc['cli']])
clw_std = np.mean(input_data[:, loc['clw']])
rh_std = np.mean(input_data[:, loc['rh']])

**Relative Humidity**

In [11]:
t0 = time.time()

# Runs for 80 mins, with all data! For 17 minutes with 10**8 samples.
rh_vals = np.linspace(np.min(input_data[:, loc['rh']]), 1.1, 40)

# sample_indices = find_closest_samples('rh', (rh_vals - rh_mean)/rh_std, 100) # The Bottleneck! Scales with O(len(rh_vals)).
# output_mean_data = [np.mean(output_data[sample_indices[:, k]]) for k in range(len(rh_vals))]

output_sfs_nn_24 = []
for rh in rh_vals:
    input_arr = np.expand_dims((np.mean(input_data, axis=0) - mean)/std, 0)
    input_arr[0, loc['rh']] = (rh - mean[loc['rh']])/std[loc['rh']]
    pred = b(sfs_24_nn.predict(input_arr))
    output_sfs_nn_24.append(pred)

# To be able to save the numbers
plot_values = {}
plot_values['RH values'] = list(rh_vals)
for eq_num in ['1', '1_mod', '4', '4_mod']:
    plot_values['pysr_EQ%s'%eq_num] = list(b(locals()['pysr_EQ%s'%eq_num](rh_vals, ta_mean, rh_z_mean, cli_mean, clw_mean)))
plot_values['sfs_nn_24'] = list(np.float64(np.array(output_sfs_nn_24)[:,0,0]))
# plot_values['data_mean'] = list(np.float64(output_mean_data))
    
# plt.plot(rh_vals, plot_values['pysr_EQ1'], color=CB_color_cycle[0])
# plt.plot(rh_vals, plot_values['pysr_EQ1_mod'], color=CB_color_cycle[1])
plt.plot(rh_vals, plot_values['pysr_EQ4'], color=CB_color_cycle[0])
plt.plot(rh_vals, plot_values['pysr_EQ4_mod'], color=CB_color_cycle[1])
plt.plot(rh_vals, plot_values['sfs_nn_24'], color=CB_color_cycle[2])
# plt.plot(rh_vals, plot_values['data_mean'], color='black')
plt.title('b) $f_{mean}$(RH)')
# plt.legend(['PySR equation', 'PySR eq., PC$_3$ enforced', 'SFS NN 24', 'DYAMOND cl_area'])
plt.legend(['PySR equation', 'PySR eq., PC$_3$ enforced', 'SFS24 NN'])
plt.xlabel('Relative Humidity')
plt.ylabel('Cloud Cover [%]')
plt.savefig('~/workspace_icon-ml/symbolic_regression/evaluate_schemes/analyzing_eqns/v1/RH_vs_cl_area_mod.pdf')

# Restore default figsize
plt.close()
plt.clf()
plt.figure()

time.time() - t0

72.66198778152466

In [11]:
with open(os.path.join(abspath, 'RH_plot_values_mod.json'), 'w') as file:
    json.dump(plot_values, file)

**Temperature**

In [12]:
ta_vals = np.linspace(np.min(input_data[:, loc['ta']]), np.max(input_data[:, loc['ta']]), 40)

sample_indices = find_closest_samples('ta', (ta_vals - ta_mean)/ta_std, 100)
output_mean_data = [np.mean(output_data[sample_indices[:, k]]) for k in range(len(ta_vals))]

output_sfs_nn_24 = []
for ta in ta_vals:
    input_arr = np.expand_dims((np.mean(input_data, axis=0) - mean)/std, 0)
    input_arr[0, loc['ta']] = (ta - mean[loc['ta']])/std[loc['ta']]
    pred = b(sfs_24_nn.predict(input_arr))
    output_sfs_nn_24.append(pred)
    
# To be able to save the numbers
plot_values = {}
plot_values['T values'] = list(ta_vals)
for eq_num in ['1', '1_mod', '4', '4_mod']:
    plot_values['pysr_EQ%s'%eq_num] = list(b(locals()['pysr_EQ%s'%eq_num](rh_mean, ta_vals, rh_z_mean, cli_mean, clw_mean)))
plot_values['sfs_nn_24'] = list(np.float64(np.array(output_sfs_nn_24)[:,0,0]))
plot_values['data_mean'] = list(np.float64(output_mean_data))
    
# plt.plot(ta_vals, plot_values['pysr_EQ1'], color=CB_color_cycle[0])
# plt.plot(ta_vals, plot_values['pysr_EQ1_mod'], color=CB_color_cycle[1])
# plt.plot(ta_vals, plot_values['pysr_EQ4'], color=CB_color_cycle[2])
plt.plot(ta_vals, plot_values['pysr_EQ4_mod'], color=CB_color_cycle[0])
plt.plot(ta_vals, plot_values['sfs_nn_24'], color=CB_color_cycle[1])
plt.plot(ta_vals, plot_values['data_mean'], color='black')
# plt.title('Based on means')
plt.legend(['PySR eq. (PC$_3$ enforced)', 'SFS NN 24', 'DYAMOND cl_area'])
plt.xlabel('Temperature [K]')
plt.ylabel('Cloud Cover')
plt.savefig('~/workspace_icon-ml/symbolic_regression/evaluate_schemes/analyzing_eqns/T_vs_cl_area_mod.pdf')

# Restore default figsize
plt.close()
plt.clf()
plt.figure()

<Figure size 640x480 with 0 Axes>

In [13]:
with open(os.path.join(abspath, 'T_plot_values_mod.json'), 'w') as file:
    json.dump(plot_values, file)

**Relative Humidity and Temperature**

In [20]:
# Increase the general font size
matplotlib.rcParams['legend.fontsize'] = 'x-large'
matplotlib.rcParams['axes.labelsize'] = 'x-large' # For an axes xlabel and ylabel
matplotlib.rcParams['axes.titlesize'] = 'x-large'
matplotlib.rcParams['xtick.labelsize'] = 'xx-large'
matplotlib.rcParams['ytick.labelsize'] = 'xx-large'
ax.set_title('title', fontsize=1)

from pylab import meshgrid,cm,imshow,contour,clabel,colorbar,axis,title,show

# Define the bounds
rh_min = np.min(input_data[:, loc['rh']])
rh_max = np.max(input_data[:, loc['rh']])
ta_min = np.min(input_data[:, loc['ta']])
ta_max = np.max(input_data[:, loc['ta']])

# Number of pixels in x- and y-direction
N_x = 30
N_y = 30
 
x = np.linspace(rh_min, rh_max, N_x)
y = np.linspace(ta_min, ta_max, N_y)
X,Y = meshgrid(x, y)

plt.figure(figsize=(14, 10))
plt.subplots_adjust(hspace=0.3, wspace=0.01)

axes = (0,1,2,3)
for i, title_name in enumerate(['PySR equation', 'PySR equation, PC$_3$ enforced', 'SFS NN 24', 'DYAMOND cloud area fraction']):
    # Add new subplot iteratively
    ax = plt.subplot(2, 2, i + 1)
    axis = axes[:i] + axes[(i+1):]
    
    ## Evaluation of the function on the grid
    # if i == 0:
    #     Z = b(locals()['pysr_EQ1'](X, Y, rh_z_mean, cli_mean, clw_mean))
    # elif i == 1:
    #     Z = b(locals()['pysr_EQ1_mod'](X, Y, rh_z_mean, cli_mean, clw_mean))
    if i == 0:
        Z = b(locals()['pysr_EQ4'](X, Y, rh_z_mean, cli_mean, clw_mean))
    elif i == 1:
        Z = b(locals()['pysr_EQ4_mod'](X, Y, rh_z_mean, cli_mean, clw_mean))
    elif i == 2:
        # Take data-means and scale by mean/std of the NN
        input_arr = (np.mean(input_data, axis=0) - mean)/std
        # Initialize mesh for every feature with its scaled mean
        input_mesh = np.ones((len(mean), N_x, N_y))
        for k in range(len(input_arr)):
            input_mesh[k] = input_arr[k]*np.ones((N_x, N_y))
        # Substitute average RH and temperature by scaled X,Y - values
        input_mesh[loc['rh']] = (X - mean[loc['rh']])/std[loc['rh']]
        input_mesh[loc['ta']] = (Y - mean[loc['ta']])/std[loc['ta']]
        # Reshape to predict
        input_mesh_rs = input_mesh.reshape(24, -1).T
        Z = b(sfs_24_nn.predict(input_mesh_rs))
        # Reshape back to plot
        Z = Z.T.reshape(N_x, N_y)
    elif i == 3:     
        step_rh = x[1] - x[0]
        inds_rh = np.round((input_data[:, loc['rh']] - rh_min)/step_rh)

        step_ta = y[1] - y[0]
        inds_ta = np.round((input_data[:, loc['ta']] - ta_min)/step_ta)
        
        Z = np.zeros((N_x, N_y))
        for M in range(N_x):
            for N in range(N_y):
                data_in_box = output_data[np.intersect1d(np.where(inds_rh == M)[0], np.where(inds_ta == N)[0])]
                if len(data_in_box) > 1000:
                    Z[M, N] = np.mean(data_in_box)
                else:
                    Z[M,N] = None
        # It looks like we have to transpose the output here
        Z = Z.T
                
    title(title_name)
    im = ax.imshow(Z, vmin=0, vmax=100, cmap='Blues_r') 

    ## Replace tick labels by proper tick labels. It's a bit cumbersome with imshow.
    # f_x = max(np.arange(0, N_x, N_x/5))/100 # The last tick is not necessarily at the end of the axis
    # f_y = max(np.arange(0, N_y, N_y/5))/100 # The last tick is not necessarily at the end of the axis
    plt.xticks(np.arange(0, N_x, N_x/5), ['%.1f'%val for val in np.linspace(min(x), max(x), 5)])
    plt.yticks(np.arange(0, N_y, N_y/5), ['%d'%val for val in np.linspace(min(y), max(y), 5)])

    # Adding the Contour lines with labels
    cset = ax.contour(Z, np.arange(10,100,40),linewidths=3,cmap=cm.Set2)
    ax.clabel(cset,inline=True,fmt='%1.1f',fontsize=10)
    colorbar(im) # adding the colobar on the right
    # # latex fashion title
    if i in [2,3]:
        plt.xlabel('Relative Humidity')
    if i in [0,2]:
        plt.ylabel('Temperature [K]')
    # show()
    
plt.savefig('~/workspace_icon-ml/symbolic_regression/evaluate_schemes/analyzing_eqns/rh_and_T_vs_cl_area_mod.pdf')

# Restore default figsize
plt.close()
plt.clf()
plt.figure()
matplotlib.rcParams.update(matplotlib.rcParamsDefault)

**Cloud Ice**

In [15]:
cli_vals = np.linspace(np.min(input_data[:, loc['cli']]), np.max(input_data[:, loc['cli']]), 40)

sample_indices = find_closest_samples('cli', (cli_vals - cli_mean)/cli_std, 100)
output_mean_data = [np.mean(output_data[sample_indices[:, k]]) for k in range(len(cli_vals))]

output_sfs_nn_24 = []
for cli in cli_vals:
    input_arr = np.expand_dims((np.mean(input_data, axis=0) - mean)/std, 0)
    input_arr[0, loc['cli']] = (cli - mean[loc['cli']])/std[loc['cli']]
    pred = b(sfs_24_nn.predict(input_arr))
    output_sfs_nn_24.append(pred)
    
# To be able to save the numbers
plot_values = {}
plot_values['cli values'] = list(cli_vals)
for eq_num in ['1', '1_mod', '4', '4_mod']:
    plot_values['pysr_EQ%s'%eq_num] = list(b(locals()['pysr_EQ%s'%eq_num](rh_mean, ta_mean, rh_z_mean, cli_vals, clw_mean)))
plot_values['sfs_nn_24'] = list(np.float64(np.array(output_sfs_nn_24)[:,0,0]))
plot_values['data_mean'] = list(np.float64(output_mean_data))
    
# plt.plot(cli_vals, plot_values['pysr_EQ1'], color=CB_color_cycle[0])
# plt.plot(cli_vals, plot_values['pysr_EQ1_mod'], color=CB_color_cycle[1])
# plt.plot(cli_vals, plot_values['pysr_EQ4'], color=CB_color_cycle[2])
plt.plot(cli_vals, plot_values['pysr_EQ4_mod'], color=CB_color_cycle[0])
plt.plot(cli_vals, plot_values['sfs_nn_24'], color=CB_color_cycle[1])
plt.plot(cli_vals, plot_values['data_mean'], color='black')
# plt.title('Based on means')
plt.legend(['PySR eq. (PC$_3$ enforced)', 'SFS NN 24', 'DYAMOND cl_area'])
plt.xlabel('Cloud Ice [kg/kg]')
plt.ylabel('Cloud Cover')
plt.savefig('~/workspace_icon-ml/symbolic_regression/evaluate_schemes/analyzing_eqns/qi_vs_cl_area_mod.pdf')

# Restore default figsize
plt.close()
plt.clf()
plt.figure()

<Figure size 640x480 with 0 Axes>

In [16]:
with open(os.path.join(abspath, 'qi_plot_values_mod.json'), 'w') as file:
    json.dump(plot_values, file)

**Cloud Water**

In [17]:
clw_vals = np.linspace(np.min(input_data[:, loc['clw']]), np.max(input_data[:, loc['clw']]), 40)

sample_indices = find_closest_samples('clw', (clw_vals - clw_mean)/clw_std, 100)
output_mean_data = [np.mean(output_data[sample_indices[:, k]]) for k in range(len(clw_vals))]

output_sfs_nn_24 = []
for clw in clw_vals:
    input_arr = np.expand_dims((np.mean(input_data, axis=0) - mean)/std, 0)
    input_arr[0, loc['clw']] = (clw - mean[loc['clw']])/std[loc['clw']]
    pred = b(sfs_24_nn.predict(input_arr))
    output_sfs_nn_24.append(pred)
    
# To be able to save the numbers
plot_values = {}
plot_values['qc values'] = list(clw_vals)
for eq_num in ['1', '1_mod', '4', '4_mod']:
    plot_values['pysr_EQ%s'%eq_num] = list(b(locals()['pysr_EQ%s'%eq_num](rh_mean, ta_mean, rh_z_mean, cli_mean, clw_vals)))
plot_values['sfs_nn_24'] = list(np.float64(np.array(output_sfs_nn_24)[:,0,0]))
plot_values['data_mean'] = list(np.float64(output_mean_data))
    
# plt.plot(clw_vals, plot_values['pysr_EQ1'], color=CB_color_cycle[0])
# plt.plot(clw_vals, plot_values['pysr_EQ1_mod'], color=CB_color_cycle[1])
# plt.plot(clw_vals, plot_values['pysr_EQ4'], color=CB_color_cycle[2])
plt.plot(clw_vals, plot_values['pysr_EQ4_mod'], color=CB_color_cycle[0])
plt.plot(clw_vals, plot_values['sfs_nn_24'], color=CB_color_cycle[1])
plt.plot(clw_vals, plot_values['data_mean'], color='black')
# plt.title('Based on means')
plt.legend(['PySR eq. (PC$_3$ enforced)', 'SFS NN 24', 'DYAMOND cl_area'])
plt.xlabel('Cloud Water [kg/kg]')
plt.ylabel('Cloud Cover')
plt.savefig('~/workspace_icon-ml/symbolic_regression/evaluate_schemes/analyzing_eqns/qc_vs_cl_area_mod.pdf')

# Restore default figsize
plt.close()
plt.clf()
plt.figure()

<Figure size 640x480 with 0 Axes>

In [18]:
with open(os.path.join(abspath, 'qc_plot_values_mod.json'), 'w') as file:
    json.dump(plot_values, file)

**Cloud Water and Ice**

In [43]:
# Increase the general font size
matplotlib.rcParams['legend.fontsize'] = 'x-large'
matplotlib.rcParams['axes.labelsize'] = 'x-large' # For an axes xlabel and ylabel
matplotlib.rcParams['axes.titlesize'] = 'x-large'
matplotlib.rcParams['xtick.labelsize'] = 'xx-large'
matplotlib.rcParams['ytick.labelsize'] = 'xx-large'
ax.set_title('title', fontsize=1)

from pylab import meshgrid,cm,imshow,contour,clabel,colorbar,axis,title,show

# Define the bounds
clw_min = np.min(input_data[:, loc['clw']])
clw_max = np.max(input_data[:, loc['clw']])
cli_min = np.min(input_data[:, loc['cli']])
cli_max = np.max(input_data[:, loc['cli']])

# Number of pixels in x- and y-direction
N_x = 30
N_y = 30
 
# MAX DIVIDED BY 3
x = np.linspace(clw_min, clw_max/3, N_x)
y = np.linspace(cli_min, cli_max/3, N_y)
X,Y = meshgrid(x, y)

plt.figure(figsize=(14, 10))
plt.subplots_adjust(hspace=0.3, wspace=0.01)

axes = (0,1,2,3)
for i, title_name in enumerate(['PySR equation', 'PySR equation, PC$_3$ enforced', 'SFS NN 24', 'DYAMOND cloud area fraction']):
    # Add new subplot iteratively
    ax = plt.subplot(2, 2, i + 1)
    axis = axes[:i] + axes[(i+1):]
    
    ## Evaluation of the function on the grid
    # if i == 0:
    #     Z = b(locals()['pysr_EQ1'](X, Y, rh_z_mean, cli_mean, clw_mean))
    # elif i == 1:
    #     Z = b(locals()['pysr_EQ1_mod'](X, Y, rh_z_mean, cli_mean, clw_mean))
    if i == 0:
        Z = b(locals()['pysr_EQ4'](rh_mean, ta_mean, rh_z_mean, Y, X))
    elif i == 1:
        Z = b(locals()['pysr_EQ4_mod'](rh_mean, ta_mean, rh_z_mean, Y, X))
    elif i == 2:
        # Take data-means and scale by mean/std of the NN
        input_arr = (np.mean(input_data, axis=0) - mean)/std
        # Initialize mesh for every feature with its scaled mean
        input_mesh = np.ones((len(mean), N_x, N_y))
        for k in range(len(input_arr)):
            input_mesh[k] = input_arr[k]*np.ones((N_x, N_y))
        # Substitute average RH and temperature by scaled X,Y - values
        input_mesh[loc['clw']] = (X - mean[loc['clw']])/std[loc['clw']]
        input_mesh[loc['cli']] = (Y - mean[loc['cli']])/std[loc['cli']]
        # Reshape to predict
        input_mesh_rs = input_mesh.reshape(24, -1).T
        Z = b(sfs_24_nn.predict(input_mesh_rs))
        # Reshape back to plot
        Z = Z.T.reshape(N_x, N_y)
    elif i == 3:     
        step_clw = x[1] - x[0]
        inds_clw = np.round((input_data[:, loc['clw']] - clw_min)/step_clw)

        step_cli = y[1] - y[0]
        inds_cli = np.round((input_data[:, loc['cli']] - cli_min)/step_cli)
        
        Z = np.zeros((N_x, N_y))
        for M in range(N_x):
            for N in range(N_y):
                data_in_box = output_data[np.intersect1d(np.where(inds_clw == M)[0], np.where(inds_cli == N)[0])]
                if len(data_in_box) > 1000:
                    Z[M, N] = np.mean(data_in_box)
                else:
                    Z[M,N] = None
        # It looks like we have to transpose the output here
        Z = Z.T
                
    title(title_name)
    im = ax.imshow(Z, vmin=0, vmax=100, cmap='Blues_r') 

    ## Replace tick labels by proper tick labels. It's a bit cumbersome with imshow.
    # f_x = max(np.arange(0, N_x, N_x/5))/100 # The last tick is not necessarily at the end of the axis
    # f_y = max(np.arange(0, N_y, N_y/5))/100 # The last tick is not necessarily at the end of the axis
    plt.xticks(np.arange(0, N_x, N_x/5), ['%.1g'%val for val in np.linspace(min(x), max(x), 5)])
    plt.yticks(np.arange(0, N_y, N_y/5), ['%.1g'%val for val in np.linspace(min(y), max(y), 5)])

    # Adding the Contour lines with labels
    cset = ax.contour(Z, np.arange(10,100,40),linewidths=3,cmap=cm.Set2)
    ax.clabel(cset,inline=True,fmt='%1.1f',fontsize=10)
    colorbar(im) # adding the colobar on the right
    # # latex fashion title
    if i in [2,3]:
        plt.xlabel('Cloud Water [kg/kg]')
    if i in [0,2]:
        plt.ylabel('Cloud Ice [kg/kg]')
    # show()
    
plt.savefig('~/workspace_icon-ml/symbolic_regression/evaluate_schemes/analyzing_eqns/clw_and_cli_vs_cl_area_mod.pdf')

# Restore default figsize
plt.close()
plt.clf()
plt.figure()
matplotlib.rcParams.update(matplotlib.rcParamsDefault)

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>